In [3]:
import os
import shutil
from diarize import (
    srt_to_transcript,
    split_waveform_filenames,
    split_waveform_speechmaps,
    split_waveform_filenames_JSON,
)
from transcriptTools import getTranscriptFiles, transcript_to_srt, reverseLookupJohn

ROOT = os.getcwd()

transcriptDir = os.path.join(ROOT, "johnTranscripts")

files = getTranscriptFiles(transcriptDir)

reversedLabels = reverseLookupJohn("speaker_labels.csv")

jsonDir = os.path.join(ROOT, "json")
os.makedirs(jsonDir, exist_ok=True)
os.makedirs(os.path.join(jsonDir, "rotl"), exist_ok=True)
os.makedirs(os.path.join(jsonDir, "roadwork"), exist_ok=True)

In [9]:
speechmapDir = os.path.join(ROOT, "speechmaps")
os.makedirs(jsonDir, exist_ok=True)
os.makedirs(os.path.join(speechmapDir, "rotl"), exist_ok=True)
os.makedirs(os.path.join(speechmapDir, "roadwork"), exist_ok=True)

for file in files:
    filepath, showname, filename = file
    episode = filename.split("_-_")[0]
    transcript = srt_to_transcript(filepath)
    unlabeledSpeaker = reversedLabels[showname][episode]
    unlabeled = [
        (idx, start, end, unlabeledSpeaker, speech)
        for idx, start, end, speaker, speech in transcript
    ]
    outpath = os.path.join(
        ROOT, "speechmaps", showname, filename.replace(".srt", ".txt")
    )
    speechmap = split_waveform_speechmaps(
        os.path.join("audio", showname, episode, filename.replace(".srt", ".wav")),
        unlabeled,
        showname,
        episode,
    )
    f = open(outpath, "w")
    f.write("\n".join(speechmap))
    f.close()

In [6]:
for file in files:
    filepath, showname, filename = file
    episode = filename.split("_-_")[0]
    transcript = srt_to_transcript(filepath)
    unlabeledSpeaker = reversedLabels[showname][episode]
    unlabeled = [
        (idx, start, end, unlabeledSpeaker, speech)
        for idx, start, end, speaker, speech in transcript
    ]
    outpath = os.path.join(ROOT, "json", showname, filename.replace(".srt", ".json"))
    json = split_waveform_filenames_JSON(
        os.path.join("audio", showname, episode, filename.replace(".srt", ".wav")),
        unlabeled,
        showname,
        episode,
    )
    f = open(outpath, "w")
    f.write(json)
    f.close()

In [15]:
johnWavDir = os.path.join(ROOT, "johnWavs")
os.makedirs(johnWavDir, exist_ok=True)
os.makedirs(os.path.join(johnWavDir, "rotl"), exist_ok=True)
os.makedirs(os.path.join(johnWavDir, "roadwork"), exist_ok=True)

for file in files:
    filepath, showname, filename = file
    episode = filename.split("_-_")[0]
    os.makedirs(os.path.join(johnWavDir, showname, episode), exist_ok=True)
    transcript = srt_to_transcript(filepath)
    unlabeledSpeaker = reversedLabels[showname][episode]
    unlabeled = [
        (idx, start, end, unlabeledSpeaker, speech)
        for idx, start, end, speaker, speech in transcript
    ]
    wavfiles = split_waveform_filenames(
        os.path.join(ROOT, "split_wavs", showname, episode),
        unlabeled,
        showname,
        episode,
    )
    for wavfile in wavfiles:
        outpath = wavfile.replace("split_wavs", "johnWavs")
        shutil.copyfile(wavfile, outpath)